In [ ]:
import folium
from folium import plugins

# 대구광역시 중심 좌표 설정 (중심부의 평균 좌표)
daegu_map = folium.Map(location=[35.8714, 128.6014], zoom_start=12)

# 예시 데이터
gas_stations = [
    {"name": "주유소1", "lat": 35.8714, "lon": 128.6014},
    {"name": "LPG 충전소1", "lat": 35.8614, "lon": 128.6114},
]

gas_stations = 

# 주유소와 충전소 위치 및 반경 표시
for station in gas_stations:
    folium.Circle(
        location=(station["lat"], station["lon"]),
        radius=500,  # 반경 500m
        color="blue" if "주유소" in station["name"] else "red",
        fill=True,
        fill_opacity=0.2,
    ).add_to(daegu_map)
    folium.Marker(
        location=(station["lat"], station["lon"]),
        popup=station["name"],
        icon=folium.Icon(color="blue" if "주유소" in station["name"] else "red", icon="info-sign"),
    ).add_to(daegu_map)

In [23]:
import pandas as pd
import folium
from IPython.display import IFrame
import json
import geopandas as gpd
from shapely.geometry import shape

# CSV 파일 읽기
point_data = pd.read_csv("주유소LPG/구별LPG충전소_위도경도.csv")

center = [35.8714, 128.6014]  # 대구광역시 중심 좌표
# 지도 생성 (대구 지역 중심)
m = folium.Map(location=center, zoom_start=18, tiles='OpenStreetMap')

# GeoJSON 데이터 로드
with open('daegu_geojson_filtered.geojson', 'r', encoding='utf-8') as f:
    daegu_geojson = json.load(f)

# point_data = pd.read_csv("경로당_수성구서구.csv")
    
# 구별 색상 설정
import random
districts = {feature['properties']['sggnm']: f"#{random.randint(0, 0xFFFFFF):06x}" for feature in daegu_geojson['features']}

# GeoJSON 데이터 지도에 추가
for feature in daegu_geojson['features']:
    gu_name = feature['properties']['sggnm']
    folium.GeoJson(
        feature,
        style_function=lambda x, gu_name=gu_name: {
            'fillColor': districts[gu_name],  # 구별 색상
            'color': 'black',  # 테두리 색
            'fillOpacity': 0.3,  # 투명도
            'weight': 1  # 테두리 두께
        },
        tooltip=f"구역: {gu_name}"  # 구 이름 표시
    ).add_to(m)

# 데이터프레임의 각 위치에 CircleMarker 및 반경 100미터 추가
for _, row in point_data.iterrows():
    # 빨간 점 (CircleMarker)
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=5,  # 원의 반지름
        color='red',  # 테두리 색상
        fill=True,  # 내부를 채움
        fill_color='red',  # 내부 색상
        fill_opacity=0.5,  # 내부 투명도
        popup=f"{row['CHARGE_NM']} ({row['ADDR']})"  # 팝업 정보
    ).add_to(m)

    # 반경 100미터 원 (Circle)
    folium.Circle(
        location=[row['위도'], row['경도']],
        radius=500,  # 반경 100미터
        color='blue',  # 테두리 색상
        fill=False,  # 내부를 채움
#         fill_color='blue',  # 내부 색상
        fill_opacity=0.2  # 내부 투명도
    ).add_to(m)

# 대구 지역 범위에 지도 초점 맞춤
bounds = [[point_data['위도'].min(), point_data['경도'].min()], [point_data['위도'].max(), point_data['경도'].max()]]
m.fit_bounds(bounds)

# 지도 저장 및 표시
map_file_path = 'daegu_map.html'
m.save(map_file_path)

# Jupyter에서 지도 표시
IFrame(map_file_path, width=1000, height=600)

In [24]:
from folium.plugins import MarkerCluster

lat = point_data['위도'].mean()
long = point_data['경도'].mean()

map_b = folium.Map((lat, long), zoom_start = 11)
mc = MarkerCluster()

for row in point_data.itertuples():
    mc.add_child(folium.Marker(location = [row.위도, row.경도]))
    map_b.add_child(mc)


map_b